# Human-to-machine-readable LSST Data Products Schema Converter
<small>v0.1, proof-of-concept</small>

Extract schemas from human-readable sources stored in Google Docs 
(or similar), into a more structured machine readable
format that can be used to generate SQL DDL statements,
etc.

## Data Model of the Spreadsheets

- Schemas are stored in a spreadsheet structure convertable
  to .csv format.

- The first column (the 'marker') is used to mark which rows
  have semantic meaning within this spec. The `marker` MUST
  be equal to 'TABLE', 'HEADER', 'COLUMN', or be empty. Any
  rows where the `marker` is empty will be ignored.
  
- A row with `marker=TABLE` signifies the start of a table
  definition. The second column in that row specifies the
  table name. Other columns are ignored; this allows additional
  human-readable data to be added to this row.
  
  Example:
  ```
     "TABLE","SSObject","LSST-computed per-object quantities. 1:1 relationship with MPCORB. Recomputed daily, upon MPCORB ingestion.",""
  ```

- The first row with a non-empty `marker` column following the
  `marker=TABLE` row MUST have `marker=HEADERS` (the "headers"
  row). The headers row names
  the DDL-related data to follow. Spreadsheet columns with
  header names not listed in `valid_names` are ignored; this
  allows additional human-readable columns to be maintained 
  with the data (an example may be the pipeline that originates
  a certain column, or various comments).
  
  Example:
  ```
      "HEADERS","Column name","Type","Not NULL","unit","UCD","Description","Origin","","","",""
  ```

- The `marker=HEADERS` row MUST be followed by one or more
  `marker=COLUMN` rows. Each of these rows defines the properties
  of a column in the current table, with the properties corresponding
  to those defined by the headers row.
  
  Example:
  ```
      "COLUMN","MOIDTrueAnomaly","FLOAT","","deg","","True anomaly of the MOID point","SSOCP","","","",""
  ```

- Multiple tables can be given in a single spreadsheet by repeating
  the table structure above.

- The format can easily be extended to capture relationships, 
  indices and constraints by adding further `marker` keywords.

## Daily Usage

In daily usage, the source-of-truth table schemas are be maintained in 
a spreadsheet tool (e.g., Google Docs). The first -- `marker` -- column
is kept hidden so as not to confuse the user and/or clutter the workspace.

- A human maintainer can easily maintain insight into the schema.
- It can be edited with equal ease (subject to usual change control requirements).
- It can be shared with other stakeholders w/o a need for special tools or
  understanding of formats.
- Derived formats (e.g., SQL DML, [the proposed YAML serialization](https://gist.github.com/brianv0/b9f3d8c0e4bc61899293816f2eb16ff1), etc.) can be extracted by running tools
  reading the data model defined above.

A working example can be seen at:

   https://docs.google.com/spreadsheets/d/1E0rTlvuJC0CvpLNsuWLK0x70uhpZww4v6GB5QkiQr-Q

## Core Reader Code

In [1]:
import pandas as pd

valid_headers = ['Column name', 'Type', 'Not NULL', 'unit', 'UCD', 'Description']

def extract_tables(df):
    """
    Given a DataFrame with holding data that follows the data model described above,
    extract the individual table schemas and yield them to the caller.
    """
    df = df.copy()
    df.columns = ["marker"] + list(range(1, len(df.columns)))
    df = df[df["marker"].notnull()]

    table_list = df[df.marker == "TABLE"]
    for iloc, (start, table) in enumerate(table_list.iterrows()):
        # Extract the [start, end] index range for the table
        table_name = table.iloc[1]
        try:
            end = table_list.index[iloc+1]-1
        except IndexError:
            end = df.index[-1]

        # Extract table headers
        headers = df.loc[start+1]
        assert(headers['marker'] == "HEADERS")
        headers = headers[headers.notnull() & headers.isin(valid_headers)]

        # Extract table data
        table=df[headers.index].loc[start+2:end]
        table.columns = headers.values
        table = table.fillna('').reset_index(drop=True)

        yield (table_name, table)

def read_google_csv(url):
    # Input example: https://docs.google.com/spreadsheets/d/1E0rTlvuJC0CvpLNsuWLK0x70uhpZww4v6GB5QkiQr-Q
    # For URL formatting spec, see https://stackoverflow.com/questions/33713084/download-link-for-google-spreadsheets-csv-export-with-multiple-sheets
    url += "/gviz/tq?tqx=out:csv"
    return pd.read_csv(url, header=None)

## Load the example dataset from Google Spreadsheets (Solar System Data Product tables)

In [2]:
import os.path
try:
    df = pd.read_csv('_cache.csv', header=None)
except FileNotFoundError:
    df = read_google_csv("https://docs.google.com/spreadsheets/d/1E0rTlvuJC0CvpLNsuWLK0x70uhpZww4v6GB5QkiQr-Q")
    df.to_csv('_cache.csv', index=False, header=False) # cache it

tables = dict( extract_tables(df) )
tables.keys()

dict_keys(['MPCORB', 'MPCORBDESIGMAP', 'SSObject', 'SSSource'])

## Example 1: Pretty-print a table

In [3]:
def show_table(tables, name):
    from IPython.display import display, Markdown

    display(Markdown('### ' + name))
    display(tables[name])

show_table(tables, 'SSSource')

### SSSource

,Column name,Type,Not NULL,unit,UCD,Description
0,ssObjectId,BIGINT,y,,meta.id;src,Unique identifier of the object.
1,diaSourceId,BIGINT,y,,meta.id;src,Unique identifier of the observation
2,mpcUniqueId,BIGINT,,,,MPC unique identifier of the observation
3,nearbyObj,BIGINT[6],,,,Closest Objects (3 stars and 3 galaxies) in Le...
4,nearbyObjDist,FLOAT[6],,,,Distances to nearbyObj
5,nearbyObjLnP,FLOAT[6],,,,Natural log of the probability that the observ...
6,eclipticLambda,DOUBLE,y,deg,,Ecliptic longitude
7,eclipticBeta,DOUBLE,y,deg,,Ecliptic latitude
8,galacticL,DOUBLE,y,deg,,Galactic longitude
9,galacticB,DOUBLE,y,deg,,Galactic latitute


## Example 2: Create corresponding DDL statements

Note how these are created in a database-agnostic way, and can be rendered to any SQL dialect that SQLAlchemy supports.

### Helper functions

In [4]:
import sqlalchemy as sa
import re

_re_type_to_satype = re.compile(r'^([A-Za-z]+)(\((\d+)\))*(\[(\d+)\])*$')
_dct_type_to_satype = {
#   <typename>  <SA type>    <is_array?>
    'INTEGER':  (sa.Integer,    False),
    'INT':      (sa.Integer,    False),
    'BIGINT':   (sa.BigInteger, False),
    'FLOAT':    (sa.Float,      False),
    'DOUBLE':   (sa.Float(53),  False),
    'VARCHAR':  (sa.String,     True),
    'STRING':   (sa.String,     True),
    'DATETIME': (sa.DateTime,   False),
    'BLOB':     (sa.BLOB,       True),
}

def type_to_satype(type_):
    # Helper to convert from textual type representation,
    # to the corresponding SqlAlchemy type object. Includes
    # an extension where a type can be suffixed by `[N]` to
    # indicate this column should be repeated N times (an
    # array).
    m = _re_type_to_satype.match(type_)
    if m is None:
        raise Exception("Could not parse type spec '%s'" % type_)

    ty, arg, repcount = m.group(1), m.group(3), m.group(5)
    ty = ty.upper()
    if arg is not None:
        arg = int(arg)
    if repcount is not None:
        repcount = int(repcount)

    saty, expectArg = _dct_type_to_satype[ty]
    if expectArg:
        saty = saty(arg)
    
    return saty, repcount

def to_sa_schema(tables):
    # Construct the SQLAlchemy MetaData object given the input CSV dataframe
    metadata = sa.MetaData()
    for table_name, tbl in tables.items():
        table = sa.Table(table_name, metadata)
        for _, row in tbl.iterrows():
            name, type_, notnull = row['Column name'], row['Type'], row['Not NULL']
            satype, repcount = type_to_satype(type_)

            # if repcount is not None, replicate columns repcount times,
            # with suffixes [1..repcount]
            suffixes = map(str, range(1, repcount+1)) if repcount is not None else ['']
            for suffix in suffixes:
                col = sa.Column(name + suffix, satype, nullable= notnull != 'y')
                table.append_column(col)
    return metadata

### Writing CREATE TABLE statements (MySQL Dialect)

In [5]:
metadata = to_sa_schema(tables)

for table in metadata.sorted_tables:
    from sqlalchemy.dialects import mysql
    from sqlalchemy.schema import CreateTable
    print(CreateTable(table).compile(dialect=mysql.dialect()))


CREATE TABLE `MPCORB` (
	`mpcDesignation` VARCHAR(8) NOT NULL, 
	`mpcNumber` INTEGER, 
	`ssObjectId` BIGINT NOT NULL, 
	`mpcH` FLOAT, 
	`mpcG` FLOAT, 
	epoch FLOAT(53), 
	`M` FLOAT(53), 
	peri FLOAT(53), 
	node FLOAT(53), 
	incl FLOAT(53), 
	e FLOAT(53), 
	n FLOAT(53), 
	a FLOAT(53), 
	`uncertaintyParameter` VARCHAR(1), 
	reference VARCHAR(9), 
	nobs INTEGER, 
	nopp INTEGER, 
	arc FLOAT, 
	`arcStart` DATETIME, 
	`arcEnd` DATETIME, 
	rms FLOAT, 
	`pertsShort` VARCHAR(3), 
	`pertsLong` VARCHAR(3), 
	computer VARCHAR(10), 
	flags INTEGER, 
	`fullDesignation` VARCHAR(26), 
	`lastIncludedObservation` FLOAT
)



CREATE TABLE `MPCORBDESIGMAP` (
	`mpcDesignation` VARCHAR(8), 
	`mpcNumber` INTEGER, 
	`otherDesignation` VARCHAR(8), 
	`ssObjectId` BIGINT
)



CREATE TABLE `SSObject` (
	`ssObjectId` BIGINT NOT NULL, 
	`discoverySubmissionDate` FLOAT(53), 
	`firstObservationDate` FLOAT(53), 
	arc FLOAT NOT NULL, 
	`numObs` INTEGER NOT NULL, 
	`lcPeriodic` BLOB(768), 
	`MOID` FLOAT, 
	`MOIDTrueAnom

### A better way of doing the same thing (SQLite dialect, for variety...)

In [6]:
# Dump the complete schema creation DDL using a trick described at
# https://docs.sqlalchemy.org/en/latest/faq/metadata_schema.html#how-can-i-get-the-create-table-drop-table-output-as-a-string
# and at https://stackoverflow.com/a/46669075
# This has the advantage it dumps the whole creation script, and in the proper order.

def dump(sql, *multiparams, **params):
    print(sql.compile(dialect=engine.dialect))

engine = sa.create_engine('sqlite://', strategy='mock', executor=dump)
metadata.create_all(engine, checkfirst=False)


CREATE TABLE "MPCORB" (
	"mpcDesignation" VARCHAR(8) NOT NULL, 
	"mpcNumber" INTEGER, 
	"ssObjectId" BIGINT NOT NULL, 
	"mpcH" FLOAT, 
	"mpcG" FLOAT, 
	epoch FLOAT, 
	"M" FLOAT, 
	peri FLOAT, 
	node FLOAT, 
	incl FLOAT, 
	e FLOAT, 
	n FLOAT, 
	a FLOAT, 
	"uncertaintyParameter" VARCHAR(1), 
	reference VARCHAR(9), 
	nobs INTEGER, 
	nopp INTEGER, 
	arc FLOAT, 
	"arcStart" DATETIME, 
	"arcEnd" DATETIME, 
	rms FLOAT, 
	"pertsShort" VARCHAR(3), 
	"pertsLong" VARCHAR(3), 
	computer VARCHAR(10), 
	flags INTEGER, 
	"fullDesignation" VARCHAR(26), 
	"lastIncludedObservation" FLOAT
)



CREATE TABLE "MPCORBDESIGMAP" (
	"mpcDesignation" VARCHAR(8), 
	"mpcNumber" INTEGER, 
	"otherDesignation" VARCHAR(8), 
	"ssObjectId" BIGINT
)



CREATE TABLE "SSObject" (
	"ssObjectId" BIGINT NOT NULL, 
	"discoverySubmissionDate" FLOAT, 
	"firstObservationDate" FLOAT, 
	arc FLOAT NOT NULL, 
	"numObs" INTEGER NOT NULL, 
	"lcPeriodic" BLOB, 
	"MOID" FLOAT, 
	"MOIDTrueAnomaly" FLOAT, 
	"MOIDEclipticLongitude" FLOAT, 

## Example 3: Dumping the schema in YAML format

This is left as an exercise for the reader.